# Start!

## Problem:
Mamy doczynienia z serią danych numerycznych wraz z problemem klasyfikacji binarnej.

Na podstawie opisów cech można wywnioskować, że są one w jakimś stopniu ze sobą powiązane - chociażby szerokość serca czy płuc z ich polem powierzchni.

### Moja początkowa intuicja:
1. Dokonać analizy PCA na cechach, by ograniczyć szum informacji
2. Zastosować klasyfikację knn z cross-examination na hiperparametrze k, by uniknąć under lub overfittingu

In [77]:
import pandas as pd
import torch
import numpy as np
from numpy import dtype

In [92]:

from torch.utils.data import Dataset

class HeartParametersDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file, sep=",")
        # Zamień przecinki na kropki w całym dataframe (dotyczy tylko stringów)
        df = df.map(lambda x: str(x).replace(",", ".") if isinstance(x, str) else x)
        # Zamień wszystko (oprócz pierwszej kolumny, tj. ID, i ew. etykiety) na float
        df = df.drop(columns=["ID"]).astype(float)

        self.labels = [torch.tensor(x) for x in df["Cardiomegaly"].values]
        self.data =  [torch.tensor(x) for x in df.drop(columns=["Cardiomegaly"]).values]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


dataset = HeartParametersDataset("task_data.csv")



(tensor([ 1.7200e+02,  4.0500e+02,  4.2469e-01,  1.6824e+03,  3.1537e+03,
         -6.3853e+02, -3.0424e-01,  6.8819e+01,  2.1345e-01,  6.7949e+02,
          2.4898e+04,  7.5419e+04], dtype=torch.float64),
 tensor(0., dtype=torch.float64))

In [73]:


U, S, V = torch.pca_lowrank(dataset.data, q=5, center=True)

projected_data = torch.matmul(dataset.data, V[:, :5])

reduced_dataset = torch.utils.data.TensorDataset(projected_data)
reduced_loader = DataLoader(reduced_dataset, batch_size=32)

print(reduced_loader[0])

TypeError: expected Tensor but got <class 'numpy.ndarray'>